# 1 IMPORTS

## 1.1 Helper Functions

## 1.2 Loading data

# 2 DESCRICAO DOS DADOS

## 2.1 Rename Columns

## 2.2 Data Dimensions

## 2.3 Data Types

## 2.4 Check NA

## 2.5 Fillout NA

## 2.6 Change Data Types

## 2.7 Descriptive Statistics

### 2.7.1 Numerical Atributes

### 2.7.2 Categorical Atributes

# 3 FEATURE ENGINEERING

## 3.1 Mapa Mental de Hipoteses

## 3.2 Criacao das Hipoteses

## 3.3 Lista Final de Hipóteses

## 3.4 Feature Engineering

# 4 FILTRAGEM DE VARIÁVEIS

## 4.1 Filtragem das Linhas


## 4.2 Selecao das Colunas

# 5 ANALISE EXPLORATORIA DOS DADOS

## 5.1 Analise Univariada

### 5.1.1 Response Variable

### 5.1.2 Numerical Variable

### 5.1.3 Categorical Variable

## 5.2 Analise Bivariada

## 5.3 Analise Multivariada

### 5.3.1 Numerical Attributes

### 5.3.2 Categorical Attributes

# 6 DATA PREPARATION

## 6.1 Normalizacao

## 6.2 Rescaling

## 6.3 Transformacao

### 6.3.1 Encoding

### 6.3.2 Response Variable Transformation

### 6.3.3 Nature Transformation

# 7 FEATURE SELECTION

## 7.1 Split dataframe into training and test dataset

## 7.2 Boruta as Feature Selector

### 7.2.1 Best Features from Boruta

## 7.3 Manual Feature Selection

# 8 MACHINE LEARNING MODELLING

## 8.1 Average Model

## 8.2 Linear Regression Model

### 8.2.1 Linear Regression Model - Cross Validation

## 8.3 Linear Regression Regularized Model - Lasso

###  8.3.1 Lasso - Cross Validation

## 8.4 Random Forest Regressor

### 8.4.1 Random Forest Regressor - Cross Validation

## 8.5 XGBoost Regressor

### 8.5.1 XGBoost Regressor - Cross Validation

## 8.6 Compare Model's Performance

### 8.6.1 Single Performance

### 8.6.2 Real Performance - Cross Validation

# 9 HYPERPARAMETER FINE TUNING

## 9.1 Random Search

## 9.2 Final Model

# 10 TRADUCAO E INTERPRETACAO DO ERRO

## 10.1 Business Performance

## 10.2 Total Performance

## 10.3 Machine Learning Performance

# 11 DEPLOY MODEL TO PRODUCTION

## 11.1 Rossmann Class

## 11.2 API Handler

## 11.3 API Tester